In [254]:
import pandas as pd
import re
import numpy as np
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

In [255]:
# Cargar los archivos CSV
df_maestro = pd.read_csv("L_STMaestroPrd.csv")  # Tu archivo maestro
# Copias de los DataFrames
df_productos = pd.read_csv("L_STMaestroPrd.csv")
copy_dfProductos = df_productos.loc[:, df_productos.notna().any()].copy()  # Eliminar columnas vacías
df_productos



C:\Users\OPERADOR\AppData\Local\Temp\ipykernel_11044\2257514907.py:2: DtypeWarning: Columns (7,14,17,18,21,28,39,42,46) have mixed types. Specify dtype option on import or set low_memory=False.
  df_maestro = pd.read_csv("L_STMaestroPrd.csv")  # Tu archivo maestro
C:\Users\OPERADOR\AppData\Local\Temp\ipykernel_11044\2257514907.py:4: DtypeWarning: Columns (7,14,17,18,21,28,39,42,46) have mixed types. Specify dtype option on import or set low_memory=False.
  df_productos = pd.read_csv("L_STMaestroPrd.csv")


,niprod,cod_prod,cod_abrev,descrip1,descrip2,abrev,cod_catprod,codbarra,cod_fbarra,unid_v,...,atrib0,atrib1,atrib2,atrib3,atrib4,atrib5,atrib6,atrib7,atrib8,atrib9
0,122703.0,MEE-EST-00904,12185,-,-,-,EM,NaN,NaN,UN,...,JAYOR,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,134925.0,APA-BAL-03444,-,-,-,-,MP,NaN,NaN,UN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,124372.0,APE-00080,SUPP500FRE,(REPETIDO) SUPPORTAN 500ML,CAJA X 15 SACHET X 500ML - UM: SACHET,SUPP500FRE,AL,NaN,NaN,UN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,125531.0,DEC-PGM-03064,48457,BOLSAS PAPEL GRADO MEDICO 175X65X420MM - KIMS,BOLSA X 500 UNIDADES UM: BOLSA,PAPEL GRADOMEDICO 17,MP,NaN,NaN,UN,...,ETILEN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,133202.0,DEC-PGM-03110,EPBL-19,BOLSAS PAPEL GRADO MEDICO 260X60X520MM - KIMS,PACK X 1000 UNIDADES UM: PACK,16/08/24,MP,NaN,NaN,UN,...,ETILEN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56769,118010.0,49702,49702,ZYVALIX 250 mg comp.x 120,NaN,ZYVALIX 250 mg comp.,PT,7795367001885.0,LKM_B,UN,...,LKM ONCO/ESPECI,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
56770,129759.0,61433,61433,ZYVALIX PLUS 500 mg comp.x 60,NaN,ZYVALIX PLUS 500 mg,PT,7795367552196.0,NaN,UN,...,LKM ONCO/ESPECI,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
56771,26456.0,27450,27450,ZYVOX IV 600 mg bolsas x 10x300ml,CAJA X 10 BOLSAS X 300ML - UM: CAJA,ZYVOX IV 600 mg bols,PT,7791824117007.0,BOEHRINGER,UN,...,PFIZER,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
56772,26457.0,27451,27451,ZYVOX ORAL 600 mg tab.x 10,NaN,ZYVOX ORAL 600 mg ta,PT,7791824116994.0,NaN,UN,...,PFIZER,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [256]:
# Este modulo selecciona que algoritmo usar para estandarizar los csv que provienen del proveedor.

# Función para procesar el caso de "varios productos en una fila"
def procesar_varios_productos(df):
    for column in df.columns:
        df[column] = df[column].apply(lambda x: x.split('\n') if isinstance(x, str) else ([x] if pd.notna(x) else [np.nan]))
    df_exploded_list = []
    for _, row in df.iterrows():
        row_lists = [row[col] if isinstance(row[col], list) else [row[col]] for col in df.columns]
        max_len = max(len(lst) for lst in row_lists)
        row_lists = [lst + [None] * (max_len - len(lst)) for lst in row_lists]
        exploded_row = pd.DataFrame(row_lists).T
        exploded_row.columns = df.columns
        df_exploded_list.append(exploded_row)
    return pd.concat(df_exploded_list, ignore_index=True)

# Función para procesar el caso de "un producto por fila"
def procesar_un_producto(df):
    for col in df.columns:
        df[col] = df[col].apply(lambda x: x.replace('\r\n', ' ').replace('\n', ' ') if isinstance(x, str) else x)
    return df

# Función principal para decidir el módulo a usar
def procesar_csv(file_path):
    df = pd.read_csv(file_path)
    contiene_saltos = df.iloc[:, 0].apply(lambda x: '\n' in x if isinstance(x, str) else False).any()
    if contiene_saltos:
        print("Usando el módulo: VARIOS PRODUCTOS EN UNA FILA")
        return procesar_varios_productos(df)
    else:
        print("Usando el módulo: UN PRODUCTO POR FILA")
        return procesar_un_producto(df)

# Cargar el CSV del proveedor
ruta_csv = "prueba3.csv"
df_resultado_PROVEEDOR = procesar_csv(ruta_csv)

Usando el módulo: UN PRODUCTO POR FILA


In [257]:
# Función de limpieza básica de texto para la columna de descripciones
def clear_column(column):
    columna_limpia = re.sub(r'\b(\d+|mg|comp\.?|blisters?|estuche|x|ml|unidades|marca|botella|caja|referencia|cm|g|m|f\.a\.x|'
                            r'jbe\.|sol\.|env\.|lata|vial|pvo\.|sobres|gotero|cápsulas|frascos?|blister|bot\.|mcg|sp|p\.b\.)\b',
                            '', column, flags=re.IGNORECASE)
    columna_limpia = re.sub(r'\n', ' ', columna_limpia)
    columna_limpia = re.sub(r'[:\.\-,;()]', '', columna_limpia)
    columna_limpia = re.sub(r'\s+', ' ', columna_limpia).strip()
    return columna_limpia


In [258]:
# Limpiamos las descripciones de los productos
df_resultado_PROVEEDOR['descripcion_limpia'] = df_resultado_PROVEEDOR['DESCRIPCIÓN'].fillna('').apply(clear_column)


In [259]:
df_resultado_PROVEEDOR["descripcion_unida"] = df_resultado_PROVEEDOR["DESCRIPCIÓN"] + " " + df_resultado_PROVEEDOR["PRESENTACION"].fillna('')
df_resultado_PROVEEDOR

,N° CERTIF.,CODIGO,CODIGO DE BARRAS,PRINCIPIO ACTIVO,DESCRIPCIÓN,PRESENTACION,PSL(s/IVA),PSL(c/IVA),PVP,descripcion_limpia,descripcion_unida
0,47497,ARUCAG010A1,7798333230081,TIROFIBAN,AGRASTAT,"0,25 mg/ml vial x 50 ml.",$ 285.074,$ 344.939,$ 500.161,AGRASTAT,"AGRASTAT 0,25 mg/ml vial x 50 ml."
1,56612,7059731,7795381000208,EPLERENONA,ALDACTONE EP,25 mg x 30 comprimidos,$ 15.143,$ 18.323,$ 26.569,ALDACTONE EP,ALDACTONE EP 25 mg x 30 comprimidos
2,56612,7059733,7795381000215,EPLERENONA,ALDACTONE EP,50 mg x 30 comprimidos,$ 22.150,$ 26.801,$ 38.862,ALDACTONE EP,ALDACTONE EP 50 mg x 30 comprimidos
3,27761,ARPEAD302N1,7798333230050,METILDOPA,ALDOMET,250 mg comp. rec. X 30,$ 7.646,$ 9.251,$ 13.414,ALDOMET,ALDOMET 250 mg comp. rec. X 30
4,27761,ARPEAD305J1,7798333230074,METILDOPA,ALDOMET,500 mg comp. rec. X 30,$ 10.987,$ 13.294,$ 19.276,ALDOMET,ALDOMET 500 mg comp. rec. X 30
...,...,...,...,...,...,...,...,...,...,...,...
63,49538,7059736,7795381410250,ZIPRASIDONA,ZELDOX,60 mg x 30 cápsulas,$ 73.519,$ 88.958,$ 128.989,ZELDOX,ZELDOX 60 mg x 30 cápsulas
64,49538,7059737,7795381410267,ZIPRASIDONA,ZELDOX,80 mg x 30 cápsulas,$ 92.681,$ 112.144,$ 162.609,ZELDOX,ZELDOX 80 mg x 30 cápsulas
65,42749,7059487,7795381410106,SERTRALINA,ZOLOFT,100 mg x 30 tabletas,$ 23.891,$ 28.908,$ 41.917,ZOLOFT,ZOLOFT 100 mg x 30 tabletas
66,42749,7059488,7795381407106,SERTRALINA,ZOLOFT,50 mg. x 30 tabletas,$ 13.291,$ 16.082,$ 23.319,ZOLOFT,ZOLOFT 50 mg. x 30 tabletas


In [260]:
# Función para filtrar productos por laboratorio
def laboratorio_input_df(df, columna):
    input_name = input("Ingrese el nombre del laboratorio: ").strip()
    tablas_coincidencias = df[df[columna].str.contains(input_name, case=False, na=False)].copy()
    if not tablas_coincidencias.empty:
        print(f"Se encontraron {len(tablas_coincidencias)} productos coincidentes del laboratorio '{input_name}'.")
        tablas_coincidencias['descrip1_limpia'] = tablas_coincidencias['descrip1'].fillna('').apply(clear_column)
        return tablas_coincidencias
    else:
        print(f"No se encontraron productos del laboratorio '{input_name}'.")
        return None

column_name_laboratorio = "atrib0"
laboratorio_coincidencias_df = laboratorio_input_df(df_productos, column_name_laboratorio)
laboratorio_coincidencias_df

Se encontraron 80 productos coincidentes del laboratorio 'ASPEN PHARMA'.


,niprod,cod_prod,cod_abrev,descrip1,descrip2,abrev,cod_catprod,codbarra,cod_fbarra,unid_v,...,atrib1,atrib2,atrib3,atrib4,atrib5,atrib6,atrib7,atrib8,atrib9,descrip1_limpia
1290,133323.0,MEE-EST-02669,24823,AGRASTAT 0.25 mg/ml (12.5mg) vial x 50 ml (PH)...,CAJA x 1 F.A. - UM:CAJA,ult.actual.03/09/24,MP,7798333230081,NaN,UN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,AGRASTAT / 5mg PH ASPEN
1291,120259.0,24823,24823,AGRASTAT 0.25 mg/ml vial x 50 ml,NaN,AGRASTAT 0.25 mg/ml,PT,7798333230081,NaN,UN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,AGRASTAT /
1881,25879.0,47287,47287,ALDACTONE EP 25 MG comp.x 30,NaN,ALDACTONE EP 25 MG c,PT,7795381000208,NaN,UN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ALDACTONE EP
1883,25881.0,47288,47288,ALDACTONE EP 50 MG comp.x 30,NaN,ALDACTONE EP 50 MG c,PT,7795381000215,NaN,UN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ALDACTONE EP
1889,120260.0,5360,5360,ALDOMET 250 mg comp.rec.x 30,NaN,ALDOMET 250 mg comp.,PT,7798333230050,NaN,UN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ALDOMET rec
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56449,26439.0,28511,28511,ZELDOX 60 mg cáps.x 30,NaN,ZELDOX 60 mg cáps.x,PT,7795381410250.0,NaN,UN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ZELDOX cáps
56450,26440.0,28512,28512,ZELDOX 80 mg cáps.x 30,NaN,ZELDOX 80 mg cáps.x,PT,7795381410267.0,NaN,UN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ZELDOX cáps
56634,26449.0,PSI423226,23226,ZOLOFT 100 mg tab.x 30,NaN,ZOLOFT 100 mg tab.x,PSI,7795381410106.0,NaN,UN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ZOLOFT tab
56638,26453.0,PSI417683,17683,ZOLOFT 50 mg tab.ran.x 30,NaN,ZOLOFT 50 mg tab.ran,PSI,7795381407106.0,NaN,UN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ZOLOFT tabran


In [261]:
laboratorio_coincidencias_df ["descripcion_unida_MASTERPRODUCTOS"] = laboratorio_coincidencias_df["descrip1"] + " " + laboratorio_coincidencias_df["descrip2"].fillna('')
laboratorio_coincidencias_df

,niprod,cod_prod,cod_abrev,descrip1,descrip2,abrev,cod_catprod,codbarra,cod_fbarra,unid_v,...,atrib2,atrib3,atrib4,atrib5,atrib6,atrib7,atrib8,atrib9,descrip1_limpia,descripcion_unida_MASTERPRODUCTOS
1290,133323.0,MEE-EST-02669,24823,AGRASTAT 0.25 mg/ml (12.5mg) vial x 50 ml (PH)...,CAJA x 1 F.A. - UM:CAJA,ult.actual.03/09/24,MP,7798333230081,NaN,UN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,AGRASTAT / 5mg PH ASPEN,AGRASTAT 0.25 mg/ml (12.5mg) vial x 50 ml (PH)...
1291,120259.0,24823,24823,AGRASTAT 0.25 mg/ml vial x 50 ml,NaN,AGRASTAT 0.25 mg/ml,PT,7798333230081,NaN,UN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,AGRASTAT /,AGRASTAT 0.25 mg/ml vial x 50 ml
1881,25879.0,47287,47287,ALDACTONE EP 25 MG comp.x 30,NaN,ALDACTONE EP 25 MG c,PT,7795381000208,NaN,UN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ALDACTONE EP,ALDACTONE EP 25 MG comp.x 30
1883,25881.0,47288,47288,ALDACTONE EP 50 MG comp.x 30,NaN,ALDACTONE EP 50 MG c,PT,7795381000215,NaN,UN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ALDACTONE EP,ALDACTONE EP 50 MG comp.x 30
1889,120260.0,5360,5360,ALDOMET 250 mg comp.rec.x 30,NaN,ALDOMET 250 mg comp.,PT,7798333230050,NaN,UN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ALDOMET rec,ALDOMET 250 mg comp.rec.x 30
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56449,26439.0,28511,28511,ZELDOX 60 mg cáps.x 30,NaN,ZELDOX 60 mg cáps.x,PT,7795381410250.0,NaN,UN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ZELDOX cáps,ZELDOX 60 mg cáps.x 30
56450,26440.0,28512,28512,ZELDOX 80 mg cáps.x 30,NaN,ZELDOX 80 mg cáps.x,PT,7795381410267.0,NaN,UN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ZELDOX cáps,ZELDOX 80 mg cáps.x 30
56634,26449.0,PSI423226,23226,ZOLOFT 100 mg tab.x 30,NaN,ZOLOFT 100 mg tab.x,PSI,7795381410106.0,NaN,UN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ZOLOFT tab,ZOLOFT 100 mg tab.x 30
56638,26453.0,PSI417683,17683,ZOLOFT 50 mg tab.ran.x 30,NaN,ZOLOFT 50 mg tab.ran,PSI,7795381407106.0,NaN,UN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ZOLOFT tabran,ZOLOFT 50 mg tab.ran.x 30


In [262]:
# Instanciamos el modelo de SentenceTransformer para obtener embeddings de texto
model = SentenceTransformer('paraphrase-MiniLM-L6-v2')

# Función para obtener embeddings de las descripciones
def get_embeddings(descriptions):
    return model.encode(descriptions.tolist())

# Comparar descripciones de productos utilizando embeddings
"""
    DESCRIPCIÓN: proviene del CSV PROVEEDOR 
    descrip1: proviene del CSV FILTRADO DE LA MASTER
"""

if laboratorio_coincidencias_df is not None:
    proveedor_embeddings = get_embeddings(df_resultado_PROVEEDOR['descripcion_limpia'])
    laboratorio_embeddings = get_embeddings(laboratorio_coincidencias_df['descrip1_limpia'])

    matches = []

    for i, prov_row in df_resultado_PROVEEDOR.iterrows():
        prov_desc = prov_row['descripcion_limpia']

        # Calculamos la similitud de coseno entre el producto proveedor y los productos del laboratorio
        cosine_similarities = cosine_similarity([proveedor_embeddings[i]], laboratorio_embeddings)

        best_match_index = cosine_similarities.argmax()  # El índice de la mejor coincidencia
        best_match_score = cosine_similarities.max()  # El puntaje de similitud

        match_info = prov_row.to_dict()
        match_info.update(laboratorio_coincidencias_df.iloc[best_match_index].to_dict())
        match_info['similaridad'] = best_match_score
        matches.append(match_info)

    df_matches = pd.DataFrame(matches)

    # Reordenamos las columnas para que la columna 'similaridad' esté al inicio
    cols = ['similaridad'] + [col for col in df_matches.columns if col != 'similaridad']
    df_matches = df_matches[cols]

    print(f"\nCoincidencias encontradas: {len(df_matches[df_matches['similaridad'] > 0])}")
    print(df_matches)

    productos_sin_coincidencia = laboratorio_coincidencias_df[~laboratorio_coincidencias_df['descrip1_limpia'].isin(
        df_matches[df_matches['similaridad'] > 0]['descrip1_limpia']
    )]

    print(f"\nProductos del laboratorio sin coincidencias: {len(productos_sin_coincidencia)}")
    print(productos_sin_coincidencia)
else:
    print("No se encontraron coincidencias para el laboratorio especificado.")



Coincidencias encontradas: 68
    similaridad  N° CERTIF.       CODIGO  CODIGO DE BARRAS PRINCIPIO  ACTIVO  \
0      0.958400       47497  ARUCAG010A1     7798333230081        TIROFIBAN    
1      1.000000       56612      7059731     7795381000208        EPLERENONA   
2      1.000000       56612      7059733     7795381000215        EPLERENONA   
3      0.877218       27761  ARPEAD302N1     7798333230050         METILDOPA   
4      0.877218       27761  ARPEAD305J1     7798333230074         METILDOPA   
..          ...         ...          ...               ...               ...   
63     0.766355       49538      7059736     7795381410250       ZIPRASIDONA   
64     0.766355       49538      7059737     7795381410267       ZIPRASIDONA   
65     0.830697       42749      7059487     7795381410106        SERTRALINA   
66     0.830697       42749      7059488     7795381407106        SERTRALINA   
67     0.830697       42749      7059489     7795381411745        SERTRALINA   

      DE

In [263]:
df_matches

,similaridad,N° CERTIF.,CODIGO,CODIGO DE BARRAS,PRINCIPIO ACTIVO,DESCRIPCIÓN,PRESENTACION,PSL(s/IVA),PSL(c/IVA),PVP,...,atrib2,atrib3,atrib4,atrib5,atrib6,atrib7,atrib8,atrib9,descrip1_limpia,descripcion_unida_MASTERPRODUCTOS
0,0.958400,47497,ARUCAG010A1,7798333230081,TIROFIBAN,AGRASTAT,"0,25 mg/ml vial x 50 ml.",$ 285.074,$ 344.939,$ 500.161,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,AGRASTAT /,AGRASTAT 0.25 mg/ml vial x 50 ml
1,1.000000,56612,7059731,7795381000208,EPLERENONA,ALDACTONE EP,25 mg x 30 comprimidos,$ 15.143,$ 18.323,$ 26.569,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ALDACTONE EP,ALDACTONE EP 25 MG comp.x 30
2,1.000000,56612,7059733,7795381000215,EPLERENONA,ALDACTONE EP,50 mg x 30 comprimidos,$ 22.150,$ 26.801,$ 38.862,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ALDACTONE EP,ALDACTONE EP 25 MG comp.x 30
3,0.877218,27761,ARPEAD302N1,7798333230050,METILDOPA,ALDOMET,250 mg comp. rec. X 30,$ 7.646,$ 9.251,$ 13.414,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ALDOMET rec,ALDOMET 250 mg comp.rec.x 30
4,0.877218,27761,ARPEAD305J1,7798333230074,METILDOPA,ALDOMET,500 mg comp. rec. X 30,$ 10.987,$ 13.294,$ 19.276,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ALDOMET rec,ALDOMET 250 mg comp.rec.x 30
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
63,0.766355,49538,7059736,7795381410250,ZIPRASIDONA,ZELDOX,60 mg x 30 cápsulas,$ 73.519,$ 88.958,$ 128.989,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ZELDOX cáps,ZELDOX 20 mg cáps.x 30
64,0.766355,49538,7059737,7795381410267,ZIPRASIDONA,ZELDOX,80 mg x 30 cápsulas,$ 92.681,$ 112.144,$ 162.609,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ZELDOX cáps,ZELDOX 20 mg cáps.x 30
65,0.830697,42749,7059487,7795381410106,SERTRALINA,ZOLOFT,100 mg x 30 tabletas,$ 23.891,$ 28.908,$ 41.917,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ZOLOFT tab,ZOLOFT 100 mg tab.x 30
66,0.830697,42749,7059488,7795381407106,SERTRALINA,ZOLOFT,50 mg. x 30 tabletas,$ 13.291,$ 16.082,$ 23.319,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ZOLOFT tab,ZOLOFT 100 mg tab.x 30


In [264]:
# Instanciamos el modelo de SentenceTransformer para obtener embeddings de texto
model = SentenceTransformer('paraphrase-MiniLM-L6-v2')

# Función para obtener embeddings de las descripciones
def get_embeddings(descriptions):
    return model.encode(descriptions.tolist())

# Comparar descripciones de productos utilizando embeddings
"""
    DESCRIPCIÓN: proviene del CSV PROVEEDOR 
    descrip1: proviene del CSV FILTRADO DE LA MASTER
"""

if laboratorio_coincidencias_df is not None:
    proveedor_embeddings = get_embeddings(df_resultado_PROVEEDOR['DESCRIPCIÓN'])
    laboratorio_embeddings = get_embeddings(laboratorio_coincidencias_df['descrip1'])

    matches = []

    for i, prov_row in df_resultado_PROVEEDOR.iterrows():
        prov_desc = prov_row['DESCRIPCIÓN']

        # Calculamos la similitud de coseno entre el producto proveedor y los productos del laboratorio
        cosine_similarities = cosine_similarity([proveedor_embeddings[i]], laboratorio_embeddings)

        best_match_index = cosine_similarities.argmax()  # El índice de la mejor coincidencia
        best_match_score = cosine_similarities.max()  # El puntaje de similitud

        match_info = prov_row.to_dict()
        match_info.update(laboratorio_coincidencias_df.iloc[best_match_index].to_dict())
        match_info['similaridad'] = best_match_score
        matches.append(match_info)

    df_matches_2 = pd.DataFrame(matches)

    # Reordenamos las columnas para que la columna 'similaridad' esté al inicio
    cols = ['similaridad'] + [col for col in df_matches_2.columns if col != 'similaridad']
    df_matches_2 = df_matches_2[cols]

    print(f"\nCoincidencias encontradas: {len(df_matches_2[df_matches_2['similaridad'] > 0])}")
    print(df_matches_2)

    productos_sin_coincidencia = laboratorio_coincidencias_df[~laboratorio_coincidencias_df['descrip1'].isin(
        df_matches_2[df_matches_2['similaridad'] > 0]['descrip1']
    )]

    print(f"\nProductos del laboratorio sin coincidencias: {len(productos_sin_coincidencia)}")
    print(productos_sin_coincidencia)
else:
    print("No se encontraron coincidencias para el laboratorio especificado.")

df_matches_2



Coincidencias encontradas: 68
    similaridad  N° CERTIF.       CODIGO  CODIGO DE BARRAS PRINCIPIO  ACTIVO  \
0      0.472624       47497  ARUCAG010A1     7798333230081        TIROFIBAN    
1      0.794308       56612      7059731     7795381000208        EPLERENONA   
2      0.794308       56612      7059733     7795381000215        EPLERENONA   
3      0.696773       27761  ARPEAD302N1     7798333230050         METILDOPA   
4      0.696773       27761  ARPEAD305J1     7798333230074         METILDOPA   
..          ...         ...          ...               ...               ...   
63     0.637831       49538      7059736     7795381410250       ZIPRASIDONA   
64     0.637831       49538      7059737     7795381410267       ZIPRASIDONA   
65     0.672653       42749      7059487     7795381410106        SERTRALINA   
66     0.672653       42749      7059488     7795381407106        SERTRALINA   
67     0.672653       42749      7059489     7795381411745        SERTRALINA   

      DE

,similaridad,N° CERTIF.,CODIGO,CODIGO DE BARRAS,PRINCIPIO ACTIVO,DESCRIPCIÓN,PRESENTACION,PSL(s/IVA),PSL(c/IVA),PVP,...,atrib2,atrib3,atrib4,atrib5,atrib6,atrib7,atrib8,atrib9,descrip1_limpia,descripcion_unida_MASTERPRODUCTOS
0,0.472624,47497,ARUCAG010A1,7798333230081,TIROFIBAN,AGRASTAT,"0,25 mg/ml vial x 50 ml.",$ 285.074,$ 344.939,$ 500.161,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,AGRASTAT /,AGRASTAT 0.25 mg/ml vial x 50 ml
1,0.794308,56612,7059731,7795381000208,EPLERENONA,ALDACTONE EP,25 mg x 30 comprimidos,$ 15.143,$ 18.323,$ 26.569,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ALDACTONE EP,ALDACTONE EP 25 MG comp.x 30
2,0.794308,56612,7059733,7795381000215,EPLERENONA,ALDACTONE EP,50 mg x 30 comprimidos,$ 22.150,$ 26.801,$ 38.862,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ALDACTONE EP,ALDACTONE EP 25 MG comp.x 30
3,0.696773,27761,ARPEAD302N1,7798333230050,METILDOPA,ALDOMET,250 mg comp. rec. X 30,$ 7.646,$ 9.251,$ 13.414,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ALDOMET rec,ALDOMET 250 mg comp.rec.x 30
4,0.696773,27761,ARPEAD305J1,7798333230074,METILDOPA,ALDOMET,500 mg comp. rec. X 30,$ 10.987,$ 13.294,$ 19.276,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ALDOMET rec,ALDOMET 250 mg comp.rec.x 30
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
63,0.637831,49538,7059736,7795381410250,ZIPRASIDONA,ZELDOX,60 mg x 30 cápsulas,$ 73.519,$ 88.958,$ 128.989,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ZELDOX cáps,ZELDOX 20 mg cáps.x 30
64,0.637831,49538,7059737,7795381410267,ZIPRASIDONA,ZELDOX,80 mg x 30 cápsulas,$ 92.681,$ 112.144,$ 162.609,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ZELDOX cáps,ZELDOX 20 mg cáps.x 30
65,0.672653,42749,7059487,7795381410106,SERTRALINA,ZOLOFT,100 mg x 30 tabletas,$ 23.891,$ 28.908,$ 41.917,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ZOLOFT tabran,ZOLOFT 50 mg tab.ran.x 30
66,0.672653,42749,7059488,7795381407106,SERTRALINA,ZOLOFT,50 mg. x 30 tabletas,$ 13.291,$ 16.082,$ 23.319,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ZOLOFT tabran,ZOLOFT 50 mg tab.ran.x 30


In [265]:
# Suponiendo que tienes los DataFrames df_matches_2 y df_matches
df1 = df_matches_2[["similaridad", "DESCRIPCIÓN", "descrip1"]]
df2 = df_matches[["similaridad"]]  # Asegúrate de que "similaridad" también esté entre corchetes


# Concatenar ambos DataFrames a lo largo del eje de las columnas (axis=1)
df_combined_similitud = pd.concat([df1, df2], axis=1)

# Imprimir el DataFrame combinado
df_combined_similitud


,similaridad,DESCRIPCIÓN,descrip1,similaridad
0,0.472624,AGRASTAT,AGRASTAT 0.25 mg/ml vial x 50 ml,0.958400
1,0.794308,ALDACTONE EP,ALDACTONE EP 25 MG comp.x 30,1.000000
2,0.794308,ALDACTONE EP,ALDACTONE EP 25 MG comp.x 30,1.000000
3,0.696773,ALDOMET,ALDOMET 250 mg comp.rec.x 30,0.877218
4,0.696773,ALDOMET,ALDOMET 250 mg comp.rec.x 30,0.877218
...,...,...,...,...
63,0.637831,ZELDOX,ZELDOX 20 mg cáps.x 30,0.766355
64,0.637831,ZELDOX,ZELDOX 20 mg cáps.x 30,0.766355
65,0.672653,ZOLOFT,ZOLOFT 50 mg tab.ran.x 30,0.830697
66,0.672653,ZOLOFT,ZOLOFT 50 mg tab.ran.x 30,0.830697


In [266]:
# Instanciamos el modelo de SentenceTransformer para obtener embeddings de texto
model = SentenceTransformer('paraphrase-MiniLM-L6-v2')

# Función para obtener embeddings de las descripciones
def get_embeddings(descriptions):
    return model.encode(descriptions.tolist())

# Comparar descripciones de productos utilizando embeddings
"""
    DESCRIPCIÓN: proviene del CSV PROVEEDOR 
    descrip1: proviene del CSV FILTRADO DE LA MASTER
"""

if laboratorio_coincidencias_df is not None:
    proveedor_embeddings = get_embeddings(df_resultado_PROVEEDOR['descripcion_unida'])
    laboratorio_embeddings = get_embeddings(laboratorio_coincidencias_df['descripcion_unida_MASTERPRODUCTOS'])

    matches = []

    for i, prov_row in df_resultado_PROVEEDOR.iterrows():
        prov_desc = prov_row['descripcion_unida']

        # Calculamos la similitud de coseno entre el producto proveedor y los productos del laboratorio
        cosine_similarities = cosine_similarity([proveedor_embeddings[i]], laboratorio_embeddings)

        best_match_index = cosine_similarities.argmax()  # El índice de la mejor coincidencia
        best_match_score = cosine_similarities.max()  # El puntaje de similitud

        match_info = prov_row.to_dict()
        match_info.update(laboratorio_coincidencias_df.iloc[best_match_index].to_dict())
        match_info['similaridad'] = best_match_score
        matches.append(match_info)

    df_matches_3 = pd.DataFrame(matches)

    # Reordenamos las columnas para que la columna 'similaridad' esté al inicio
    cols = ['similaridad'] + [col for col in df_matches_3.columns if col != 'similaridad']
    df_matches_3 = df_matches_3[cols]

    print(f"\nCoincidencias encontradas: {len(df_matches_3[df_matches_3['similaridad'] > 0])}")
    print(df_matches_3)

    productos_sin_coincidencia = laboratorio_coincidencias_df[~laboratorio_coincidencias_df['descripcion_unida_MASTERPRODUCTOS'].isin(
        df_matches_3[df_matches_3['similaridad'] > 0]['descripcion_unida_MASTERPRODUCTOS']
    )]

    print(f"\nProductos del laboratorio sin coincidencias: {len(productos_sin_coincidencia)}")
    print(productos_sin_coincidencia)
else:
    print("No se encontraron coincidencias para el laboratorio especificado.")




Coincidencias encontradas: 68
    similaridad  N° CERTIF.       CODIGO  CODIGO DE BARRAS PRINCIPIO  ACTIVO  \
0      0.993623       47497  ARUCAG010A1     7798333230081        TIROFIBAN    
1      0.938012       56612      7059731     7795381000208        EPLERENONA   
2      0.936309       56612      7059733     7795381000215        EPLERENONA   
3      1.000000       27761  ARPEAD302N1     7798333230050         METILDOPA   
4      1.000000       27761  ARPEAD305J1     7798333230074         METILDOPA   
..          ...         ...          ...               ...               ...   
63     0.932061       49538      7059736     7795381410250       ZIPRASIDONA   
64     0.932257       49538      7059737     7795381410267       ZIPRASIDONA   
65     0.745517       42749      7059487     7795381410106        SERTRALINA   
66     0.768827       42749      7059488     7795381407106        SERTRALINA   
67     0.763173       42749      7059489     7795381411745        SERTRALINA   

      DE

In [267]:
df_matches_3

,similaridad,N° CERTIF.,CODIGO,CODIGO DE BARRAS,PRINCIPIO ACTIVO,DESCRIPCIÓN,PRESENTACION,PSL(s/IVA),PSL(c/IVA),PVP,...,atrib2,atrib3,atrib4,atrib5,atrib6,atrib7,atrib8,atrib9,descrip1_limpia,descripcion_unida_MASTERPRODUCTOS
0,0.993623,47497,ARUCAG010A1,7798333230081,TIROFIBAN,AGRASTAT,"0,25 mg/ml vial x 50 ml.",$ 285.074,$ 344.939,$ 500.161,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,AGRASTAT /,AGRASTAT 0.25 mg/ml vial x 50 ml
1,0.938012,56612,7059731,7795381000208,EPLERENONA,ALDACTONE EP,25 mg x 30 comprimidos,$ 15.143,$ 18.323,$ 26.569,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ALDACTONE EP,ALDACTONE EP 25 MG comp.x 30
2,0.936309,56612,7059733,7795381000215,EPLERENONA,ALDACTONE EP,50 mg x 30 comprimidos,$ 22.150,$ 26.801,$ 38.862,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ALDACTONE EP,ALDACTONE EP 50 MG comp.x 30
3,1.000000,27761,ARPEAD302N1,7798333230050,METILDOPA,ALDOMET,250 mg comp. rec. X 30,$ 7.646,$ 9.251,$ 13.414,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ALDOMET rec,ALDOMET 250 mg comp.rec.x 30
4,1.000000,27761,ARPEAD305J1,7798333230074,METILDOPA,ALDOMET,500 mg comp. rec. X 30,$ 10.987,$ 13.294,$ 19.276,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ALDOMET rec,ALDOMET 500 mg comp.rec.x 30
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
63,0.932061,49538,7059736,7795381410250,ZIPRASIDONA,ZELDOX,60 mg x 30 cápsulas,$ 73.519,$ 88.958,$ 128.989,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ZELDOX cáps,ZELDOX 60 mg cáps.x 30
64,0.932257,49538,7059737,7795381410267,ZIPRASIDONA,ZELDOX,80 mg x 30 cápsulas,$ 92.681,$ 112.144,$ 162.609,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ZELDOX cáps,ZELDOX 80 mg cáps.x 30
65,0.745517,42749,7059487,7795381410106,SERTRALINA,ZOLOFT,100 mg x 30 tabletas,$ 23.891,$ 28.908,$ 41.917,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ZOLOFT tab,ZOLOFT 100 mg tab.x 30
66,0.768827,42749,7059488,7795381407106,SERTRALINA,ZOLOFT,50 mg. x 30 tabletas,$ 13.291,$ 16.082,$ 23.319,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ZOLOFT tab,ZOLOFT 100 mg tab.x 30


In [268]:
import pandas as pd

# Suponiendo que tienes los DataFrames df_matches, df_matches_2 y df_matches_3
df1 = df_matches[["similaridad"]]
df2 = df_matches_2[["similaridad"]]
df3 = df_matches_3[["similaridad", "descripcion_unida_MASTERPRODUCTOS", "descripcion_unida", "niprod"]]

# Concatenar los DataFrames a lo largo del eje de las columnas (axis=1)
df_combined_similitud = pd.concat([df1, df2, df3], axis=1)

# Renombrar las columnas de similaridad de cada DataFrame para diferenciarlas
df_combined_similitud.columns = ['similaridad_1', 'similaridad_2', "similaridad_3",  "descripcion_unida_MASTERPRODUCTOS", "descripcion_unida", "niprod"]

# Imprimir el DataFrame combinado con los nuevos nombres de columna
df_combined_similitud


,similaridad_1,similaridad_2,similaridad_3,descripcion_unida_MASTERPRODUCTOS,descripcion_unida,niprod
0,0.958400,0.472624,0.993623,AGRASTAT 0.25 mg/ml vial x 50 ml,"AGRASTAT 0,25 mg/ml vial x 50 ml.",120259.0
1,1.000000,0.794308,0.938012,ALDACTONE EP 25 MG comp.x 30,ALDACTONE EP 25 mg x 30 comprimidos,25879.0
2,1.000000,0.794308,0.936309,ALDACTONE EP 50 MG comp.x 30,ALDACTONE EP 50 mg x 30 comprimidos,25881.0
3,0.877218,0.696773,1.000000,ALDOMET 250 mg comp.rec.x 30,ALDOMET 250 mg comp. rec. X 30,120260.0
4,0.877218,0.696773,1.000000,ALDOMET 500 mg comp.rec.x 30,ALDOMET 500 mg comp. rec. X 30,120262.0
...,...,...,...,...,...,...
63,0.766355,0.637831,0.932061,ZELDOX 60 mg cáps.x 30,ZELDOX 60 mg x 30 cápsulas,26439.0
64,0.766355,0.637831,0.932257,ZELDOX 80 mg cáps.x 30,ZELDOX 80 mg x 30 cápsulas,26440.0
65,0.830697,0.672653,0.745517,ZOLOFT 100 mg tab.x 30,ZOLOFT 100 mg x 30 tabletas,26449.0
66,0.830697,0.672653,0.768827,ZOLOFT 100 mg tab.x 30,ZOLOFT 50 mg. x 30 tabletas,26449.0
